<a href="https://colab.research.google.com/github/cserock/colab-examples/blob/main/07_RAG_web_QA_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 환경변수 파일(.env) 설정하기
/content/drive/MyDrive/lg-dx/에 아래 내용으로 .env 파일을 작성한 후 업로드합니다.  

OPENAI_API_KEY=sk-xxxx  
LANGCHAIN_TRACING_V2="true"  
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"  
LANGCHAIN_API_KEY="lsv2_xxxx  
LANGCHAIN_PROJECT="lg-dx"


Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

python-dotenv 라이브러리 설치

In [ ]:
!pip install python-dotenv

환경변수 파일 로드 및 확인

In [ ]:
from dotenv import load_dotenv
load_dotenv('/content/drive/MyDrive/lg-dx/.env', override=True)

import os
openai_api_key = os.environ.get('OPENAI_API_KEY')
print("openai_api_key : " + openai_api_key)
langchain_api_key = os.environ.get('LANGCHAIN_API_KEY')
print("langchain_api_key : " + langchain_api_key)

## LangChain 설치 및 업데이트

In [ ]:
# LangChain 설치 및 업데이트
!pip install -qU langchain-teddynote langchain_community pymupdf faiss-cpu

설치된 LangChain 버전을 확인합니다.

In [ ]:
from importlib.metadata import version

print("[LangChain 관련 패키지 버전]")
for package_name in [
    "langchain",
    "langchain-core",
    "langchain-experimental",
    "langchain-community",
    "langchain-openai",
    "langchain-teddynote",
    "langchain-huggingface",
    "langchain-google-genai",
    "langchain-anthropic",
    "langchain-cohere",
    "langchain-chroma",
    "langchain-elasticsearch",
    "langchain-upstage",
    "langchain-cohere",
    "langchain-milvus",
    "langchain-text-splitters",
]:
    try:
        package_version = version(package_name)
        print(f"{package_name}: {package_version}")
    except ImportError:
        print(f"{package_name}: 설치되지 않음")

LangSmith 추적 설정  
.env 파일에 LangSmith API 키가 설정 되어 있어야 합니다.(LANGCHAIN_API_KEY)

In [ ]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("lg-dx-rag-web")

## 네이버 뉴스 기반 QA(Question-Answering) 챗봇
이번 튜토리얼에는 네이버 뉴스기사의 내용에 대해 질문할 수 있는 뉴스기사 QA 앱 을 구축할 것입니다.  

이 가이드에서는 OpenAI 챗 모델과 임베딩, 그리고 Chroma 벡터 스토어를 사용할 것입니다.  

먼저 다음의 과정을 통해 간단한 인덱싱 파이프라인과 RAG 체인을 약 20줄의 코드로 구현할 수 있습니다.
- bs4는 웹 페이지를 파싱하기 위한 라이브러리입니다.
- langchain은 AI와 관련된 다양한 기능을 제공하는 라이브러리로, 여기서는 특히 텍스트 분할(RecursiveCharacterTextSplitter), 문서 로딩(WebBaseLoader), 벡터 저장(Chroma, FAISS), 출력 파싱(StrOutputParser), 실행 가능한 패스스루(RunnablePassthrough) 등을 다룹니다.  
- langchain_openai 모듈을 통해 OpenAI의 챗봇(ChatOpenAI)과 임베딩(OpenAIEmbeddings) 기능을 사용할 수 있습니다.


In [ ]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## RAG 기본 파이프라인(1~8단계)

In [ ]:
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

In [ ]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [ ]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
# 검색 결과 4개(default)
retriever = vectorstore.as_retriever()

# 검색 결과 8개
# retriever = vectorstore.as_retriever(search_kwargs={"k": 8})


In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)


In [ ]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
answer = chain.invoke("부영그룹의 출산 장려 정책에 대해 설명해주세요.")
print(answer)

In [ ]:
answer = chain.invoke("부영그룹은 출산 직원에게 얼마의 지원을 제공하나요?")
print(answer)

In [ ]:
answer = chain.invoke("정부의 저출생 대책을 bullet points 형식으로 작성해 주세요.")
print(answer)

In [ ]:
answer = chain.invoke("부영그룹의 임직원 숫자는 몇명인가요?")
print(answer)